# Modelo de Sentimiento

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (DistilBertTokenizer, 
                          DistilBertForSequenceClassification, 
                          Trainer, 
                          TrainingArguments, 
                          pipeline)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from transformers import AdamW


## Carga de datos sintéticos

In [6]:
dataset = pd.read_parquet("hf://datasets/TimKoornstra/synthetic-financial-tweets-sentiment/data/train-00000-of-00001.parquet")
print(dataset.shape)
dataset.head()


(1428771, 2)


,tweet,sentiment
0,💰 Cashing out stocks today has left me with a ...,2
1,💸 Losing half my investments in the last month...,2
2,📉 Crypto bubble finally bursts! The crypto tra...,2
3,🔥 The blazing inferno engulfs the stock market...,2
4,🥶 My stocks are stuck in an icy grip today. Fi...,2


## Preprocesado de los datos

In [7]:
# Seleccionamos las columnas necesarias
dataset = dataset[['tweet', 'sentiment']]

# Reducir el tamaño del dataset al 5% para pruebas rápidas
reduced_dataset = dataset.sample(frac=0.05, random_state=42)

# Dividir el dataset en entrenamiento (80%) y validación (20%)
train_df, val_df = train_test_split(reduced_dataset, test_size=0.2, random_state=42)

# Convertir los DataFrames a formato Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Renombrar las columnas para adaptarlas al modelo
train_dataset = train_dataset.rename_column("sentiment", "labels")
val_dataset = val_dataset.rename_column("sentiment", "labels")

print(f"Train Dataset Shape: {train_dataset.shape}")


Train Dataset Shape: (57151, 3)


## Tokenización de los Tweets

In [8]:
# Cargar el tokenizador de DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Función de preprocesado para tokenizar los tweets
def preprocess_function(examples):
    return tokenizer(examples['tweet'], truncation=True, padding=True, max_length=128)

# Aplicar la tokenización a los datasets de entrenamiento y validación
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/57151 [00:00<?, ? examples/s]

Map:   0%|          | 0/14288 [00:00<?, ? examples/s]

## Cargar el Modelo Preentrenado

In [9]:
# Cargar DistilBERT para clasificación de secuencias (3 etiquetas de sentimiento)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3, dropout=0.3)

# Forzar el uso de GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Running in {device}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running in cuda


## Definición de los Parámetros de Entrenamiento

In [10]:
# Configuración de los parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida
    num_train_epochs=3,              # Número de épocas
    per_device_train_batch_size=32,  # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch para validación
    warmup_steps=500,                # Número de pasos de warm-up
    weight_decay=0.1,                # Decaimiento del peso
    logging_dir='./logs',            # Directorio para los logs
    logging_steps=10,                # Frecuencia de los logs
    evaluation_strategy="epoch",     # Evaluación al final de cada época
    save_strategy="epoch",           # Guardar el modelo al final de cada época
    load_best_model_at_end=True,     # Cargar el mejor modelo al final
    fp16=True                        # Precisión mixta para mayor velocidad en GPU
)

c:\Users\pmyoe\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Métricas de Evaluación

In [11]:
# Función para calcular las métricas (precisión, recall, f1)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

## Optimización y Entrenamiento

In [12]:
# Optimizador AdamW
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.1)

# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

# Entrenar el modelo
trainer.train()

c:\Users\pmyoe\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\pmyoe\AppData\Local\Temp\ipykernel_8968\1090211906.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Cargar el Mejor Modelo

In [ ]:
# Cargar el mejor checkpoint guardado durante el entrenamiento
best_checkpoint = './results/checkpoint-5358'  # Reemplaza con el mejor checkpoint
model = DistilBertForSequenceClassification.from_pretrained(best_checkpoint)
tokenizer = DistilBertTokenizer.from_pretrained(best_checkpoint)

# Crear el pipeline de clasificación
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


Tweet: The stock market is doing great today! 🚀
Predicción: Positivo (Confianza: 0.9974)

Tweet: I can't believe I lost so much money. 😔
Predicción: Negativo (Confianza: 0.9991)

Tweet: Not sure about investing anymore, the market is too volatile.
Predicción: Negativo (Confianza: 0.9993)



## Realizar Predicciones de Prueba

In [ ]:
# Ejemplo de tweets para clasificación
tweets = [
    "The stock market is doing great today! 🚀",
    "I can't believe I lost so much money. 😔",
    "Not sure about investing anymore, the market is too volatile."
]

# Obtener las predicciones
predictions = classifier(tweets)

# Mapeo de las etiquetas a los sentimientos
label_mapping = {"LABEL_0": "Neutral", "LABEL_1": "Positivo", "LABEL_2": "Negativo"}

# Mostrar los resultados de las predicciones
for tweet, pred in zip(tweets, predictions):
    sentiment = label_mapping[pred['label']]
    print(f"Tweet: {tweet}")
    print(f"Predicción: {sentiment} (Confianza: {pred['score']:.4f})\n")